<a href="https://colab.research.google.com/github/Faye912/samples/blob/main/1b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from gurobipy import Model, GRB
from itertools import permutations
from tqdm import tqdm

In [ ]:
def generate_random_binary_matrix(n, d):
    return np.random.randint(0, 2, size=(n, d))

def compute_xtx(X):
    return X.T @ X

def is_row_permutation(A, B):
    """Check if matrices A and B are row permutations of each other."""
    A_rows = [tuple(row) for row in A]
    B_rows = [tuple(row) for row in B]
    return sorted(A_rows) == sorted(B_rows)

def reconstruct_using_gurobi(M, n, d):
    model = Model()
    model.setParam('OutputFlag', 0)  # Suppress Gurobi output

    # Binary variables for X̃
    X_vars = [[model.addVar(vtype=GRB.BINARY) for j in range(d)] for i in range(n)]

    model.update()

    # Constraint: match entries of X̃ᵀ X̃ = M
    for i in range(d):
        for j in range(i, d):  # Only need upper triangle
            expr = sum(X_vars[k][i] * X_vars[k][j] for k in range(n))
            model.addConstr(expr == M[i, j])

    model.optimize()

    if model.Status == GRB.OPTIMAL:
        X_tilde = np.array([[int(X_vars[i][j].X + 0.5) for j in range(d)] for i in range(n)])
        return X_tilde
    else:
        return None  # Optimization failed

In [ ]:
# === Experiment ===
n_values = [3, 6, 12]
d_values = list(range(1, 13))
num_trials = 20

results = {}

for n in n_values:
    for d in d_values:
        success_count = 0
        for _ in range(num_trials):
            X = generate_random_binary_matrix(n, d)
            M = compute_xtx(X)
            X_tilde = reconstruct_using_gurobi(M, n, d)
            if X_tilde is not None and is_row_permutation(X, X_tilde):
                success_count += 1
        results[(n, d)] = success_count / num_trials
        print(f"n={n}, d={d} → success rate: {results[(n, d)]:.2f}")

n=3, d=1 → success rate: 1.00
n=3, d=2 → success rate: 1.00
n=3, d=3 → success rate: 1.00
n=3, d=4 → success rate: 1.00
n=3, d=5 → success rate: 1.00
n=3, d=6 → success rate: 1.00
n=3, d=7 → success rate: 1.00
n=3, d=8 → success rate: 1.00
n=3, d=9 → success rate: 1.00
n=3, d=10 → success rate: 1.00
n=3, d=11 → success rate: 1.00
n=3, d=12 → success rate: 1.00
n=6, d=1 → success rate: 1.00
n=6, d=2 → success rate: 1.00
n=6, d=3 → success rate: 0.85
n=6, d=4 → success rate: 0.85
n=6, d=5 → success rate: 0.95
n=6, d=6 → success rate: 1.00
n=6, d=7 → success rate: 1.00
n=6, d=8 → success rate: 1.00
n=6, d=9 → success rate: 1.00
n=6, d=10 → success rate: 1.00
n=6, d=11 → success rate: 1.00
n=6, d=12 → success rate: 1.00
n=12, d=1 → success rate: 1.00
n=12, d=2 → success rate: 1.00
n=12, d=3 → success rate: 0.70
n=12, d=4 → success rate: 0.45
n=12, d=5 → success rate: 0.15
n=12, d=6 → success rate: 0.30
n=12, d=7 → success rate: 0.20
n=12, d=8 → success rate: 0.30
n=12, d=9 → success rate: 

In [ ]:
# Display results as a table
df = pd.DataFrame(index=n_values, columns=d_values)
for (n, d), val in results.items():
    df.loc[n, d] = round(val, 2)
print("\nReconstruction Success Rates:")
print(df)


Reconstruction Success Rates:
     1    2     3     4     5    6    7    8    9    10   11    12
3   1.0  1.0   1.0   1.0   1.0  1.0  1.0  1.0  1.0  1.0  1.0   1.0
6   1.0  1.0  0.85  0.85  0.95  1.0  1.0  1.0  1.0  1.0  1.0   1.0
12  1.0  1.0   0.7  0.45  0.15  0.3  0.2  0.3  0.4  0.7  0.8  0.95
